# SubMe

Review the **Processing Settings** below, then click `Runtime > Run All` to start the program.



# Project Setup

It may take a few minutes to load the program. Please be patient.

In [1]:
a = 1
print(a*2)
print("hello")

2


In [ ]:
#@title Project Settings

import os

#@markdown Repo path to clone from.
repo_url = 'https://github.com/BingLingGroup/autosub' #@param {type:"string"}
repo_name = repo_url.split('/')[-1]

#@markdown Repo branch or tag to clone. Leave blank to clone `master`.
repo_branch = '' #@param {type:"string"}

#@markdown Root workspace.
root_dir = '/content/subme' #@param {type:"string"}
repo_dir = os.path.join(root_dir, repo_name)

#@markdown Temp directory. If your root workspace is in Google Drive, then this path should be a local folder, such as `/content`.
temp_dir = '/content/subme/work' #@param {type:"string"}

#@markdown Models directory.
model_dir = '/content/subme/work/checkpoint' #@param {type:"string"}

#@markdown Masks directory.
altered_dir = '/content/subme/work/altered' #@param {type:"string"}

#@markdown Output directory.
output_dir = '/content/subme/work/output' #@param {type:"string"}

#@markdown Attempt to request Google Drive access. Will mount to `/content/drive/My Drive`. 
#@markdown Will save output to your linked Google Drive account inside `subme` folder. 
#@markdown
#@markdown Try auto-mounting by clicking the Files button on the left, then click `Mount Drive`.
request_drive = False #@param {type:"boolean"} 

#@markdown Install `dependencies.txt` from project.
install_deps = True #@param {type:"boolean"}

#@#markdown Tensorflow version (1 or 2)
tf_version = 1 #@#param {type:"integer"}

#@#markdown Attempt to invoke high memory (25.5 GB) in Colab.
high_memory = False #@#param {type:"boolean"}


# Upload Videos for Processing

In [ ]:
#@#title Upload Videos for Processing
#@markdown Click `Runtime > Run All` to upload images. If you see an error in this section, then run this cell again.

from google.colab import files
import shutil

# wj faster plssss
print("wj here")

